In [1]:
import numpy as np
import tensorflow as tf
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing import image
import os
from numpy.linalg import norm
from tensorflow.keras import Sequential,layers
from annoy import AnnoyIndex
import cv2

In [2]:
ResnetModel = ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))

In [3]:
model = Sequential([
    ResnetModel,
    layers.GlobalAveragePooling2D(), #2048
])

In [4]:
def extract_features(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = model.predict(img)
    return features.flatten()

In [5]:
dataset_directory = "D:/test1/ImageRetreival/FLOWERS"

In [6]:
dataset_features = []
dataset_filenames = []
for filename in os.listdir(dataset_directory):
    image_path = os.path.join(dataset_directory, filename)
    features = extract_features(image_path)
    dataset_features.append(features)  
    dataset_filenames.append(filename)

1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 76ms/step


1/1 [==============================] - 0s 84ms/step


In [7]:
num_trees = 10  # You can adjust this parameter for trade-off between speed and accuracy
feature_dim = len(dataset_features[0])
annoy_index = AnnoyIndex(feature_dim, 'angular') #angular is for cosine similarity metric

In [8]:
for i, feature in enumerate(dataset_features):
    annoy_index.add_item(i, feature)

In [9]:
annoy_index.build(num_trees)

True

In [10]:
def Cosine_similarity(img1,img2):
    dot_product = np.dot(img1,img2.T)
    img1_norm = np.linalg.norm(img1)
    img2_norm = np.linalg.norm(img2)
    return (dot_product/(img1_norm*img2_norm))

In [11]:
query_image_path = "D:/test1/ImageRetreival/FLOWERS/daisies_00002.jpg"
query_features = extract_features(query_image_path)

1/1 [==============================] - 0s 83ms/step


In [12]:
top_n = 10
similar_indices = annoy_index.get_nns_by_vector(query_features, top_n)

In [13]:
for i, index in enumerate(similar_indices):
    similar_image_filename = dataset_filenames[index]
    similar_image_path = os.path.join(dataset_directory, similar_image_filename)
    similar_image = cv2.imread(similar_image_path)

    # Calculate the similarity using Cosine Similarity
    similarity_score = Cosine_similarity(query_features, dataset_features[index])

    print(f"Similar Image {i + 1}: Cosine Similarity - {similarity_score}")
    cv2.imshow(f"Similar Image {i + 1}", similar_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

Similar Image 1: Cosine Similarity - 1.0
Similar Image 2: Cosine Similarity - 0.9269671440124512
Similar Image 3: Cosine Similarity - 0.907619059085846
Similar Image 4: Cosine Similarity - 0.9012947082519531
Similar Image 5: Cosine Similarity - 0.8833693265914917
Similar Image 6: Cosine Similarity - 0.8677548170089722
Similar Image 7: Cosine Similarity - 0.8632436394691467
Similar Image 8: Cosine Similarity - 0.8614963889122009
Similar Image 9: Cosine Similarity - 0.8546366691589355
Similar Image 10: Cosine Similarity - 0.851381778717041
